# Solutions to Problem Set 1 — Coding Part

**Course:** Data Compression With And Without Deep probabilistic models (Prof. Bamler at University of Tübingen)

- This notebook shows suggested solutions to the coding part of Problem Set 1, published on 19 April 2023 and discussed on 26 April 2023.
- You can download the original problem set and the solutions to the other problems on it from the [course website](https://robamler.github.io/teaching/compress23/).

## Problem 1.3 (b): Huffman Coding Implementation (Encoder Part)

This problem guides you through the implementation of the Huffman Coding algorithm (Algorithm 2 on the problem set).

We represent bit strings (such as code words) as lists of boolean values, where `True` represents a "one"-bit and `False` represents a "zero" bit.
Please be aware that this would be an extremely inefficient representation for a real application, but the simplicity of this representation will make it easier to understand the algorithm.

Your task is to implement a class `HuffmanEncoder` that constructs a Huffman tree which is best suited for *encoding*.
The following is a proposal for how you could represent the Huffman tree.
Feel free to use your own representation if you have a better idea.

A Huffman tree represents a code book as a binary tree that has the symbols $x \in \mathfrak X = \{0,1,\ldots, |\mathfrak X|-1\}$ at its leaves.
Regardless of its specific shape, any binary tree with $|\mathfrak X|$ leaves has $(2|\mathfrak X| - 1)$ nodes (including the leaves; this can be shown by induction).
Thus, we can assume, without loss of generality, that the nodes (or "vertices") of the final Huffman tree are $V=\{0,1,\ldots,2|\mathfrak X|-2\}$, and that the first $|\mathfrak X|$ nodes are the leaf nodes that correspond to the symbols with the same numbers.
We therefore don't even have to explicitly represent $V$ in memory, it is already known implicitly.

Further, the final Huffman tree has $|V|-1=2|\mathfrak X| - 2$ edges because each node except for the root node has exactly one parent, to which it is connected with an edge.
We do represent the edges explicitly in memory because they define the shape of the specific Huffman tree.
But we don't represent them as a set of tuples between vertices, as one typically does in math, because this would make searching for an edge unnecessarily expensive and cumbersome.
Instead, we represent the edges as a list (named `edges`) of length $(2|\mathfrak X| - 2)$.
For each edge between a `parent` $\in V$ and a `child` $\in V$, we set `edges[child] = (parent, label)` where `label` is the label of the edge, which can be either `True` or `False`.

### Your Tasks

- Read and understand the code in the method `__getitem__` below.
  Then complete the constructor (`__init__`) by filling in all the places where it says `TODO`.
  The constructor should create the tree structure that `__getitem__` expects (as described above).
  Your implementation should follow closely Algorithm 2 on the problem set.
- Run the minimal provided unit test to make sure you didn't do any obvious mistake.
- Implement and run additional unit tests. In particular, test edge case like ties or a degenerate probability distribution (that puts all mass on a single symbol.

In [1]:
import heapq # See Problem 0.4 on Problem Set 0.

In [2]:
class HuffmanEncoder:
    """A code book for Huffman coding, optimized for *encoding*."""
    
    def __init__(self, probabilities):
        """Create a Huffman tree for symbols with the given `probabilities`.
        
        The alphabet is assumed to be {0, 1, 2, ..., len(probabilities) - 1}.

        Args:
            probabilities (list or numpy array): The probabilities of each
                symbol in the alphabet. The first entry is the probability of
                symbol `0`, the second entry is the probability of symbol `1`,
                and so on. All entries must be nonnegative and finite.
        """
        
        # Initialize the `edges` list with the correct length but empty entries.
        # We will overwrite these entries below.
        edges = [None] * (2 * len(probabilities) - 2)
        
        # TODO: initialize a binary heap called `frontier` that has entries of 
        # the form `(weight, node)`, representing the set `F` from Algorithm 2
        # on the problem set, where `node` is an index into `d`
        # (for the initial entries of `frontier`, `index` is the symbol and
        # `weight` is its probability).
        #
        # You may want to refer back to Problem 0.4 on Problem Set 0.
        #
        # (about 2-3 lines of code)
        #
        # PROPOSED SOLUTION:
        frontier = [(weight, index) for index, weight in enumerate(probabilities)]
        heapq.heapify(frontier)
        # ALTERNATIVE SOLUTION (slightly less performant):
        # frontier = []
        # for index, weight in enumerate(probabilities):
        #     heapq.heappush(frontier, (weight, index))
        
        next_node = len(probabilities)
        while len(frontier) > 1:
            # TODO: implement the rest of Algorithm 2 (about 6 lines of code)
            #
            # PROPOSED SOLUTION:
            weight1, child1 = heapq.heappop(frontier)
            weight2, child2 = heapq.heappop(frontier)
            edges[child1] = (next_node, False)
            edges[child2] = (next_node, True)
            heapq.heappush(frontier, (weight1 + weight2, next_node))
            next_node += 1

        self.edges = edges
        
    def __getitem__(self, symbol):
        """Return the code word for a given `symbol`.

        Args:
            symbol (int): A symbol from the alphabet {0, 1, ..., len-1}.
                
        Returns:
            The code word for `symbol`.
        """
        assert 0 <= symbol and symbol < len(self)
        
        codeword_reverse = []
        node = symbol
        root_node = len(self.edges)
        while node != root_node:
            node, bit = self.edges[node]
            codeword_reverse.append(bit)
        
        return list(reversed(codeword_reverse))
    
    def __len__(self):
        """Returns the size of the alphabet."""
        # As discussed above, any binary tree with `N` leaves has
        # `M = 2*N - 2` edges. Therefore, `N = M / 2 + 1`.
        return len(self.edges) // 2 + 1 # ("//" denotes integer division)

#### Unit rests:

In [3]:
def minimal_encoder_test():
    probabilities = [.3, .28, .12, .1, .2]
    expected_codebook = {
        0: [True, True],
        1: [True, False],
        2: [False, True, True],
        3: [False, True, False],
        4: [False, False],
    }
    huffman_encoder = HuffmanEncoder(probabilities)
    for symbol, expected_codeword in expected_codebook.items():
        assert huffman_encoder[symbol] == expected_codeword

minimal_encoder_test()

In [4]:
# TODO: implement more tests, in particular for edge cases